# Semantic Search dengan Sentence Transformers

Notebook ini mengimplementasikan semantic search menggunakan sentence embeddings untuk pencarian dokumen kesehatan.

## Objectives:
1. Load dan preprocess corpus
2. Generate document embeddings menggunakan multilingual model
3. Implement semantic search dengan cosine similarity
4. Evaluasi dan bandingkan dengan TF-IDF/BM25
5. Save artifacts untuk Streamlit app

## 1. Setup & Import Libraries

In [1]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from tqdm import tqdm
import json

# Impor sentence transformers
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Pra-pemrosesan
import sys
sys.path.append('..')
from src.preprocessing import preprocess_query

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Processed Data

In [3]:
# Muat data yang sudah diproses
data_path = Path("../data/processed/processed_full.csv") 
df = pd.read_csv(data_path)

print(f"Dataset dimuat: {len(df)} dokumen")
print(f"Kolom: {df.columns.tolist()}")
df.head()

Dataset dimuat: 360513 dokumen
Kolom: ['title', 'question', 'answer', 'question_date', 'answer_date', 'topics', 'topic_set', 'risk', 'year', 'time_to_answer', 'full_question', 'clean_question', 'clean_answer', 'processed_question', 'processed_answer', 'answer_count', 'full_question_length', 'answer_length', 'processed_question_length', 'processed_answer_length']


,title,question,answer,question_date,answer_date,topics,topic_set,risk,year,time_to_answer,full_question,clean_question,clean_answer,processed_question,processed_answer,answer_count,full_question_length,answer_length,processed_question_length,processed_answer_length
0,Khasiat obat zinc sulphate,Dok saya mau tanya Anak saya kan kenak fimosis...,"Hai Yevie, Terima kasih atas pertanyaannya. Zi...","23 September 2017, 18:50","24 September 2017, 10:42",zinc-sulphate,zinc-sulphate,low,2017,0.0,Khasiat obat zinc sulphate Dok saya mau tanya ...,khasiat obat zinc sulphate dok saya mau tanya ...,"hai yevie, terima kasih atas pertanyaannya. zi...",khasiat obat zinc sulphate dok mau tanya anak ...,hai yevie terima kasih atas tanya zinc sulphat...,1,293,1179,231,769
1,Perbedaan jenis formula zinc,siang dokter.... dokter sayang ingin bertanya ...,"Halo Pendys, Zinc merupakan salah satu minera...","5 August 2017, 12:16","5 August 2017, 16:27",zinc-sulphate,zinc-sulphate,low,2017,0.0,Perbedaan jenis formula zinc siang dokter.... ...,perbedaan jenis formula zinc siang dokter.... ...,"halo pendys, zinc merupakan salah satu mineral...",beda jenis formula zinc siang dokter dokter sa...,halo pendys zinc rupa salah satu mineral butuh...,1,359,1969,239,1372
2,Mengkonsumsi suplemen zinc yang sudah kadaluarsa,"Malam dok, saya baru menemukan suplemen zinc s...","Hai IriSh, Terimakasih telah bertanya ke Alodo...","12 December 2018, 20:54","13 December 2018, 17:08",suplemen zinc-sulphate,zinc-sulphate,low,2018,0.0,Mengkonsumsi suplemen zinc yang sudah kadaluar...,mengkonsumsi suplemen zinc yang sudah kadaluar...,"hai irish, terimakasih telah bertanya ke alodo...",konsumsi suplemen zinc kadaluarsa malam dok ba...,hai irish terimakasih tanya alodokter zinc zin...,1,291,1616,178,1060
3,Keamanan konsumsi suplemen zinc saat program h...,"Dear dokter, Umur saya 24 tahun dan baru menik...","Selamat pagi, terimakasih atas pertanyaannya S...","7 January 2019, 15:09","8 January 2019, 09:32",suplemen zinc-sulphate,zinc-sulphate,low,2019,0.0,Keamanan konsumsi suplemen zinc saat program h...,keamanan konsumsi suplemen zinc saat program h...,"selamat pagi, terimakasih atas pertanyaannya s...",aman konsumsi suplemen zinc program hamil dear...,selamat pagi terimakasih atas tanya suplementa...,1,491,896,319,582
4,Suplemen apa yang banyak mengandung zinc.,Sakit flu tak kunjung sembuh disebabkan karena...,"Selamat malam, terimakasih atas pertanyaannya ...","30 March 2019, 06:05","30 March 2019, 20:25",suplemen zinc-sulphate,zinc-sulphate,low,2019,0.0,Suplemen apa yang banyak mengandung zinc. Saki...,suplemen apa yang banyak mengandung zinc. saki...,"selamat malam, terimakasih atas pertanyaannya ...",suplemen apa banyak kandung zinc sakit flu tak...,selamat malam terimakasih atas tanya meski tub...,1,179,1220,122,781


## 3. Load Semantic Model

Menggunakan `paraphrase-multilingual-MiniLM-L12-v2` yang support bahasa Indonesia

In [4]:
# Muat model sentence transformer multibahasa
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"

print(f"Memuat model: {MODEL_NAME}")
print("Proses ini mungkin memakan waktu beberapa menit pada eksekusi pertama...")

model = SentenceTransformer(MODEL_NAME)

print("Model berhasil dimuat!")
print(f"Dimensi embedding: {model.get_sentence_embedding_dimension()}")

Memuat model: paraphrase-multilingual-MiniLM-L12-v2
Proses ini mungkin memakan waktu beberapa menit pada eksekusi pertama...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  36%|###5      | 168M/471M [00:00<?, ?B/s]

C:\Users\ACER NITRO 5\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ACER NITRO 5\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model berhasil dimuat!
Dimensi embedding: 384


## 4. Prepare Document Texts

Combine title + answer untuk representasi semantik yang lebih kaya

In [5]:
# Siapkan dokumen: gabungkan title dan answer
documents = []
for _, row in df.iterrows():
    title = str(row.get('title', ''))
    answer = str(row.get('answer', ''))
    combined = f"{title} {answer}"
    documents.append(combined)

print(f"Menyiapkan {len(documents)} dokumen untuk embedding")
print("\nContoh dokumen:")
print(documents[0][:300] + "...")

Menyiapkan 360513 dokumen untuk embedding

Contoh dokumen:
Khasiat obat zinc sulphate Hai Yevie, Terima kasih atas pertanyaannya. Zinc Sulphate  merupakan salah satu mineral yang dibutuhkan oleh tubuh, bermanfaat untuk membantu penyembuhan luka, memperkuat sistem kekebalan tubuh, membantu pertumbuhan sel, dan manfaat-manfaat lainnya. Jadi sebenarnya fungsi ...


## 5. Generate Document Embeddings

**⚠️ Warning**: Proses ini bisa memakan waktu tergantung ukuran corpus!

In [6]:
# Generate embeddings untuk seluruh dokumen
print("Menghasilkan embeddings...")
print("Proses ini mungkin memakan beberapa menit tergantung ukuran korpus...")

embeddings = model.encode(
    documents,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("\nEmbeddings selesai dibuat!")
print(f"   Dimensi: {embeddings.shape}")

Menghasilkan embeddings...
Proses ini mungkin memakan beberapa menit tergantung ukuran korpus...


Batches:   0%|          | 0/11267 [00:00<?, ?it/s]


Embeddings selesai dibuat!
   Dimensi: (360513, 384)
   Perkiraan ukuran memori: 528.10 MB


## 6. Implement Semantic Search Function

In [7]:
def semantic_search(query, model, embeddings, df, top_k=10):
    """
    Perform semantic search using sentence embeddings
    
    Args:
        query: Search query string
        model: SentenceTransformer model
        embeddings: Precomputed document embeddings
        df: DataFrame with document metadata
        top_k: Number of results to return
    
    Returns:
        DataFrame with top-k results and scores
    """
    # Lakukan encoding pada query
    query_embedding = model.encode([query], convert_to_numpy=True)
    
    # Hitung cosine similarity
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    
    # Ambil indeks top-k
    top_indices = similarities.argsort()[-top_k:][::-1]
    
    # Bangun dataframe hasil
    results = df.iloc[top_indices].copy()
    results['score'] = similarities[top_indices]
    results['rank'] = range(1, len(results) + 1)
    
    return results

print("Semantic search function ready!")

Semantic search function ready!


## 7. Test Semantic Search

In [8]:
# Uji query
test_query = "anak demam tinggi dan batuk"

print(f"Query: {test_query}")
print("=" * 60)

results = semantic_search(test_query, model, embeddings, df, top_k=5)

for _, row in results.iterrows():
    print(f"\n[{row['rank']}] Skor: {row['score']:.4f}")
    print(f"Title: {row['title']}")
    print(f"Answer: {row['answer'][:200]}...")
    print("-" * 60)

Query: anak demam tinggi dan batuk

[1] Skor: 0.8257
Title: Diare, dilanjutkan dengan demam naik turun dan batuk-batuk 
Answer: Alo, terimakasih atas pertanyaannya. Diare , dilanjutkan dengan  demam  naik turun dan  batuk-batuk  mungkin menandakan anak Anda mengalami: Infeksi  di sistem pencernaan, sistem pernapasan, atau sist...
------------------------------------------------------------

[2] Skor: 0.8247
Title: demam pada anak hingga muntah dan berkeringat
Answer: Halo Pak Semy,Memiliki anak yang sedang sakit tentu mengkhawatirkan orang tua.Pada dasarnya, demam merupakan salah satu pertanda adanya infeksi dalam tubuh baik infeksi virus, bakteri, atau parasit. S...
------------------------------------------------------------

[3] Skor: 0.8233
Title: Pengobatan pada demam disertai muntah
Answer: Alo Puja, Terima kasih atas pertanyaannya. Apakah ayah Anda atau Anda mengetahui penyebab dari gejala demam dan muntah yang dialami oleh ayah Anda?  Demam  merupakan salah satu respons tubuh k

## 8. Multiple Query Examples

Test dengan berbagai tipe query untuk melihat performa semantic search

In [9]:
# Uji dengan berbagai query
test_queries = [
    "sakit kepala berkepanjangan",
    "cara mengatasi flu pada bayi",
    "diet sehat untuk diabetes",
    "gejala covid-19 omicron",
]

for query in test_queries:
    results = semantic_search(query, model, embeddings, df, top_k=3)
    
    print(f"\n{'=' * 70}")
    print(f"Query: {query}")
    print('=' * 70)
    
    for _, row in results.iterrows():
        print(f"[{row['rank']}] {row['score']:.4f} - {row['title']}")
    print()


Query: sakit kepala berkepanjangan
[1] 0.8934 - Sakit kepala berkepanjangan
[2] 0.8824 - Sakit kepala berkepanjangan saat berubah posisi
[3] 0.8802 - sakit kepala yang berkepanjangan


Query: cara mengatasi flu pada bayi
[1] 0.8480 - Obat untuk mengatasi flu yang aman untuk bayi usia 6 bulan
[2] 0.8470 - Obat flu bagi ibu menyusui
[3] 0.8458 - Pengobatan pilek dan flu pada bayi usia 2 bulan


Query: diet sehat untuk diabetes
[1] 0.8464 - Diet yang baik untuk penderita diabetes
[2] 0.8440 - Keamanan konsumsi alpukat 1 buah setiap hari pada penderita diabetes
[3] 0.8372 - Menu makanan yang baik bagi penderita diabetes


Query: gejala covid-19 omicron
[1] 0.7841 - Ciri-ciri gejala covid-19
[2] 0.7765 - Hilang indra penciuman dan perasa serta batuk berdahak, apakah gejala virus corona?
[3] 0.7715 - Ciri-ciri gejala covid-19



## 9. Comparison: Semantic vs TF-IDF vs BM25

Load existing models untuk perbandingan (optional)

In [10]:
# Opsional: muat TF-IDF dan BM25 untuk perbandingan
try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    # Muat artefak TF-IDF
    with open('../artifacts/tfidf/vectorizer.pkl', 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    tfidf_matrix = pickle.load(open('../artifacts/tfidf/tfidf_matrix.pkl', 'rb'))
    
    # Muat artefak BM25
    with open('../artifacts/bm25/bm25.pkl', 'rb') as f:
        bm25_model = pickle.load(f)
    
    print("Model TF-IDF dan BM25 berhasil dimuat untuk perbandingan")
    comparison_available = True
except Exception as e:
    print(f"Tidak dapat memuat model perbandingan: {e}")
    print("Lewati perbandingan atau jalankan notebook TF-IDF/BM25 terlebih dahulu")
    comparison_available = False

Model TF-IDF dan BM25 berhasil dimuat untuk perbandingan


In [11]:
# Fungsi perbandingan
if comparison_available:
    def compare_models(query, top_k=5):
        # Model semantic
        sem_results = semantic_search(query, model, embeddings, df, top_k)
        
        # Model TF-IDF
        processed = preprocess_query(query)
        qv = tfidf_vectorizer.transform([processed])
        tfidf_sims = cosine_similarity(qv, tfidf_matrix).flatten()
        tfidf_idx = tfidf_sims.argsort()[-top_k:][::-1]
        
        # Model BM25
        tokens = processed.split()
        bm25_scores = bm25_model.get_scores(tokens)
        bm25_idx = bm25_scores.argsort()[-top_k:][::-1]
        
        print(f"Query: {query}")
        print("=" * 80)
        print(f"\n{'Rank':<6} {'Semantic':<40} {'TF-IDF':<40} {'BM25':<40}")
        print("-" * 80)
        
        for i in range(top_k):
            sem_title = df.iloc[sem_results.index[i]]['title'][:35]
            tfidf_title = df.iloc[tfidf_idx[i]]['title'][:35]
            bm25_title = df.iloc[bm25_idx[i]]['title'][:35]
            
            print(f"{i+1:<6} {sem_title:<40} {tfidf_title:<40} {bm25_title:<40}")
        
        # Hitung overlap
        sem_set = set(sem_results.index)
        tfidf_set = set(tfidf_idx)
        bm25_set = set(bm25_idx)
        
        all_overlap = sem_set & tfidf_set & bm25_set
        
        print(f"\nOverlap (ketiga model): {len(all_overlap)} dokumen")
        print(f"   Semantic ∩ TF-IDF: {len(sem_set & tfidf_set)}")
        print(f"   Semantic ∩ BM25: {len(sem_set & bm25_set)}")
        print(f"   TF-IDF ∩ BM25: {len(tfidf_set & bm25_set)}")
    
    # Jalankan perbandingan awal
    compare_models("anak demam tinggi dan batuk")

Query: anak demam tinggi dan batuk

Rank   Semantic                                 TF-IDF                                   BM25                                    
--------------------------------------------------------------------------------
1      Diare, dilanjutkan dengan demam nai      Anak demam bolehkah di suntik            Pengobatan demam tinggi pada anak d     
2      demam pada anak hingga muntah dan b      Demam tinggi pada anak                   Penyebab dan solusi batuk, pilek, d     
3      Pengobatan pada demam disertai munt      Mengatasi demam tinggi pada bayi         demam dan batuk pada anak               
4      demam naik turun disertai batuk dan      Penggunaan selimut saat anak demam       demam pada anak disertai batuk berd     
5      Pengobatan batuk disertai demam pad      demam tinggi pada anak                   demam tinggi pada anak                  

Overlap (ketiga model): 0 dokumen
   Semantic ∩ TF-IDF: 0
   Semantic ∩ BM25: 0
   TF-IDF ∩ BM25: 0


## 10. Save Artifacts for Streamlit App

In [12]:
# Buat direktori artefak semantic
SEMANTIC_DIR = Path("../artifacts/semantic")
SEMANTIC_DIR.mkdir(parents=True, exist_ok=True)

# Simpan embeddings
np.save(SEMANTIC_DIR / "embeddings.npy", embeddings)
print(f"Embeddings disimpan ke {SEMANTIC_DIR / 'embeddings.npy'}")

# Simpan metadata korpus
df_meta = df[['title', 'answer', 'topic_set', 'year']].copy()
df_meta.to_pickle(SEMANTIC_DIR / "corpus_meta.pkl")
print(f"Metadata korpus disimpan ke {SEMANTIC_DIR / 'corpus_meta.pkl'}")

# Simpan konfigurasi
config = {
    "model_name": MODEL_NAME,
    "n_docs": len(df),
    "embedding_dim": embeddings.shape[1],
    "corpus_size": len(documents)
}

with open(SEMANTIC_DIR / "semantic_config.json", "w") as f:
    json.dump(config, f, indent=2)
print(f"Konfigurasi disimpan ke {SEMANTIC_DIR / 'semantic_config.json'}")

print("\nSeluruh artefak berhasil disimpan!")
print(f"Lokasi: {SEMANTIC_DIR.absolute()}")

Embeddings disimpan ke ..\artifacts\semantic\embeddings.npy
Metadata korpus disimpan ke ..\artifacts\semantic\corpus_meta.pkl
Konfigurasi disimpan ke ..\artifacts\semantic\semantic_config.json

Seluruh artefak berhasil disimpan!
Lokasi: d:\UAS-STKI\notebook\..\artifacts\semantic


## 11. Optional: Save Model Locally

Untuk offline use, save model ke artifacts folder

In [14]:
model_path = SEMANTIC_DIR / "model"
model.save(str(model_path))
print(f"Model disimpan ke {model_path}")

Model disimpan ke ..\artifacts\semantic\model


## Summary

✅ **Semantic Search Implementation Complete!**

### What we did:
1. Loaded multilingual sentence transformer model
2. Generated embeddings for all documents (title + answer)
3. Implemented semantic search with cosine similarity
4. Tested with various health-related queries
5. (Optional) Compared with TF-IDF and BM25
6. Saved artifacts for Streamlit integration

### Artifacts saved:
- `embeddings.npy` - Document embeddings (numpy array)
- `corpus_meta.pkl` - Document metadata (pandas DataFrame)
- `semantic_config.json` - Configuration details

### Next Steps:
1. Run Streamlit app: `streamlit run app.py`
2. Select "semantic" from model dropdown
3. Try "Mode Perbandingan" to compare all models side-by-side

### Model Performance:
- **Strengths**: Understanding context, semantic similarity, paraphrase detection
- **Best for**: Natural language queries, concept-based search
- **Trade-offs**: Slower than TF-IDF/BM25, requires more memory